In [51]:
import numpy as np
import pandas as pd
from patsy import dmatrices # it fits our data into matrices
import warnings

In [52]:
# here we are defining our sigmoid fuunciton
def sigmoid (x):
    '''sigmoid function of x.'''
    return 1/(1+np.exp(-x))

In [53]:
#here we are manually defining all our hyperparameters
np.random.seed(2) # getting same random values


tol=1e-8  #convergence tolerence(it is gonna tell our model when to stop learning)

lam= None # for L2 regularization

max_iteration=20

# here we use these variables fpor creating our data

r=0.95   # covariance between x and z

n=1000   # no of observations 
sigma=1000   # variance of noise  i.e. how spread the data is

beta_z,beta_x,beta_v= -4,0.9,1 # i.e. that we know what the true data coefficient of the data are

var_x,var_z,var_v=1,1,4   # i.e. variance of inputs i.e. how spread our inputs are

# fromula spepcification for model for fititng of the model

formula ='y~x+z+v+np.exp(x)+I(v**2+z)'


In [54]:
#here we are generating our data for our model

x,z=np.random.multivariate_normal([0,0],[[var_x,r],[var_z,r]],n).T     # here n= no of observations , r=covariance 

v= np.random.normal(0,var_v,n)

A=pd.DataFrame({'x':x,'z':z,'v':v})

A['log_odds']=sigmoid(A[['x','z','v']].dot([beta_x,beta_z,beta_v])+sigma*np.random.normal(0,1,n))  # for computing log odds of our independent variables

A['y']=[np.random.normal(1,p) for p in A.log_odds]

y,X=dmatrices(formula,A,return_type='dataframe')

X.head()


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: covariance is not positive-semidefinite.
  This is separate from the ipykernel package so we can avoid doing imports until


,Intercept,x,z,v,np.exp(x),I(v ** 2 + z)
0,1.0,0.421998,0.400898,4.159684,1.525005,17.703870
1,1.0,2.163054,2.054902,5.923396,8.697664,37.141524
2,1.0,1.815984,1.725185,0.823702,6.147124,2.403670
3,1.0,-0.509204,-0.483744,1.245744,0.600974,1.068133
4,1.0,1.071254,1.017691,3.761811,2.919037,15.168915


In [55]:
def catch_singularity(f):
    '''for catching a wanring f singularioty'''
    def silencer (*args,**kwargs):
        try:
            return f(*args,**kwargs)
        except np.linalg.LinAlgError:
            warning.warn('Algorithm terminated singular Hessian')
            return args[0]
        return silencer
    

In [56]:
def newton_step(curr,X,len=None):
    '''One Naive step of newton's method'''
    
    
    # now here we are computing necessary objects
    # probability matrix
    p=np.array(sigmoid (X.dot(curr[:.0])),ndmin=2).T #ndmin= the dimensions i.e. 2 and T for the transpose
    
    #creating weight matrix
    W=np.diag((p*(1-p))[:,0])
    
    
    # we needed two things i.e. hessian and the gradient 
    
    #deriving the hessian 
    hessian =X.T.dot(W).dot(X)
    
    #deriving the gradient
    grad=X.T.dot(y-p) #     y is our predicted output p is our probability value 
    
    # regularization step 
    
    if lam:
        step ,*_=np.linalg.lstsq(hessian+ lam * np.eye(curr.shape[0]),grad)
    else:
        step,*_=np.linalg.lstsq(hessian,grad)
    beta=curr+step
    
    return beta

In [57]:
def alt_newton_step(curr,X,lam=None):
    '''one naive step of newton's method'''
    #compute necessary objects
    p=np.array(sigmoid(X.dot(curr[:,0])),ndmin=2).T
    W=np.diag((p*(1-p))[:,0])
    hessian=X.T.dot(W).dot(X)
    grad=X.T.dot(y-p)
    
    #regularization
    if lam:
        step=np.dot(np.linalg.inv(hessian+lam*np.eye(curr.shape[0])),grad)
        
    else:
        step=np.dot(np.linalg.inv(hessian),grad)
        
    # update our weights
    beta= curr+step
    return beta


In [58]:
# convergence Setup
# or the threshold to stop learning 
def check_coefs_convergence(beta_old,beta_new,tol,iters):
    '''Checks whether the coefficients have converged in thr l-infinity norm 
    Return true if they have cinverged, False Otherwise'''
    
    coef_change=np.abs(beta_old-beta_new)
    
    return not (np.any(coef_change>tol)&(iters<max_iter))

In [68]:
beta_old,beta=np.ones((len(X.columns),1)),np.zeros((len(X.columns),1))
iter_count=0
coefs_converged=False
while not coefs_converged:
    beta_old=beta
    beta=newton_step(beta,X,lam=lam)
    iter_count+=1
    coefs_converged=check_coefs_convergence(beta_old,beta,tol,iter_count)
    
print('Iteraations : {}'.format(iter_count))
print('beta : {}'.format(beta))

TypeError: newton_step() got an unexpected keyword argument 'lam'